In [1]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model

from joblib import load

from IPython.display import clear_output

import brainflow

from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds, BrainFlowError

from tqdm.notebook import tqdm


from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

%matplotlib inline

from utils.svm import preProcess_1 as preProcess_svm
from utils.deep import preProcess_1 as preProcess_deep
from utils.visualize import showMe
%load_ext autoreload
%autoreload 2


In [ ]:
svm = load('saved_models/svm_acc_95.joblib')

In [ ]:
deep = load_model('saved_models/dense_val_acc_94')

In [9]:
####      INIT BOARD        #######
BoardShim.enable_dev_board_logger()
params = BrainFlowInputParams()
board = BoardShim(BoardIds.MINDROVE_WIFI_BOARD, params)

board.prepare_session()
sample_rate = board.get_sampling_rate(16)
n_channels  = 6

sample_rate = board.get_sampling_rate(16)
size_of_ring_buffer=45000

print("Device ready (sampling rate: {}hz)".format(sample_rate))

Device ready (sample_rate: 500)


In [ ]:
def SVM_Predict(classes = ['Eyebrow','Chew']):
    classes = ['Rest'] + classes
   
    length_of_trial  = 5 #seconds
    length_of_signal = 2 #seconds
    board.start_stream((sample_rate*length_of_trial)+2) # plus 2 sec to be sure
    
    while True:
        clear_output()        
        print("Perform command!")
        
        board.get_board_data() # clear buffer
        time.sleep(length_of_signal*1.1) # record longer to make sure there is enough data
        data = board.get_board_data()
        data = data[:6,:sample_rate*length_of_signal] #keep the data of the eeg channels only, and remove data over the trial length
    
        preProcessed = preProcess_svm(data, 100)
        prediction = svm.predict(preProcessed)[0]
        print(f'Prediction: {classes[prediction]}')
    
        
SVM_Predict()


In [ ]:
def Deep_Predict(classes = ['Eyebrow','Chew']):
    classes = ['Rest'] + classes
   
    length_of_trial  = 5 #seconds
    length_of_signal = 2 #seconds
    board.start_stream((sample_rate*length_of_trial)+2) # plus 2 sec to be sure
    
    while True:
        clear_output()        
        print("Perform command!")
        
        board.get_board_data() # clear buffer
        time.sleep(length_of_signal*1.1) # record longer to make sure there is enough data
        data = board.get_board_data()
        data = data[:6,:sample_rate*length_of_signal] #keep the data of the eeg channels only, and remove data over the trial length
    
        preProcessed = preProcess_deep(data, 200)

        prediction = svm.predict([preProcessed])[0]   # Fake 1 batch 
        print(prediction)  
        prediction = classes[max(prediction)]
        print(f'Prediction: {classes[prediction]}')
    
        
Deep_Predict()

In [ ]:
board.stop_stream()
board.release_session()